In [7]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
from tensorflow.keras import optimizers, callbacks
import numpy as np
import pandas as pd
import pathlib
import os
from os import getcwd
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob
import multiprocessing
import time
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

## https://www.kaggle.com/mlg-ulb/creditcardfraud?select=creditcard.csv

## By default, a TextLineDataset yields every line of each file, including a header line. Header can be removed using the Dataset.skip() or Dataset.filter() transformations. 

In [8]:
df = pd.read_csv(r'C:\data\Data_Class\creditcard.csv', index_col=None)
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
Time      284807 non-null float64
V1        284807 non-null float64
V2        284807 non-null float64
V3        284807 non-null float64
V4        284807 non-null float64
V5        284807 non-null float64
V6        284807 non-null float64
V7        284807 non-null float64
V8        284807 non-null float64
V9        284807 non-null float64
V10       284807 non-null float64
V11       284807 non-null float64
V12       284807 non-null float64
V13       284807 non-null float64
V14       284807 non-null float64
V15       284807 non-null float64
V16       284807 non-null float64
V17       284807 non-null float64
V18       284807 non-null float64
V19       284807 non-null float64
V20       284807 non-null float64
V21       284807 non-null float64
V22       284807 non-null float64
V23       284807 non-null float64
V24       284807 non-null float64
V25       284807 non-null float64
V26  

In [10]:
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

182276 train examples
45569 validation examples
56962 test examples


## Create an input pipeline using tf.data
__Next, wrap the dataframes with tf.data.__

__This will enable us to use feature columns as a bridge to map from the columns in the Pandas dataframe to features used to train the model.__ 
__If we were working with a very large CSV file (so large that it does not fit into memory), we would use tf.data to read it from disk directly. That is not covered in this tutorial.__

In [6]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, label,shuffle=True, batch_size=32, ):
  df = dataframe.copy()
  labels = df.pop(label)
  ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(df))
  ds = ds.batch(batch_size)
  return ds

In [11]:
batch_size = 4 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train,'Class', batch_size=batch_size)
val_ds = df_to_dataset(val,'Class', shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test,'Class', shuffle=False, batch_size=batch_size)

## After creating the the input pipeline, check to see the format of the data it returns with  a small batch size.

In [13]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of Amount:', feature_batch['Amount'])
  print('A batch of Class:', label_batch )

Every feature: ['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']
A batch of Amount: tf.Tensor([ 0.76 10.   29.95 18.88], shape=(4,), dtype=float64)
A batch of Class: tf.Tensor([0 0 0 0], shape=(4,), dtype=int64)


In [16]:
df.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

In [17]:
feature_columns = []

# numeric cols
for header in ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']:
  feature_columns.append(feature_column.numeric_column(header))

In [18]:
feature_columns

[NumericColumn(key='V1', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='V2', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='V3', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='V4', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='V5', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='V6', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='V7', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='V8', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='V9', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='V10', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='V11', shape=(1,), d

## Create a feature layer, we have defined our feature columns, we will use a DenseFeatures layer to input them to our Keras model.

In [19]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [20]:
batch_size = 32
train_ds = df_to_dataset(train,'Class', batch_size=batch_size)
val_ds = df_to_dataset(val, 'Class', shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, 'Class',shuffle=False, batch_size=batch_size)

In [21]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dropout(.1),
  layers.Dense(1)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=10)

Epoch 1/10
Consider rewriting this model with the Functional API.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Consider rewriting this model with the Functional API.
5697/5697 [==============================] - ETA: 0s - loss: 0.0287 - accuracy: 0.9989WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'Time': <tf.Tensor 'ExpandDims_1:0' shape=(None, 1) dtype=float64>, 'V1': <tf.Tensor 'ExpandDims_2:0' shape=(None, 1) dtype=float64>, 'V2': <tf.Tensor 'ExpandDims_13:0' shap

5697/5697 [==============================] - 25s 4ms/step - loss: 0.0054 - accuracy: 0.9994 - val_loss: 0.0103 - val_accuracy: 0.9993
Epoch 7/10
5697/5697 [==============================] - 25s 4ms/step - loss: 0.0054 - accuracy: 0.9993 - val_loss: 0.0072 - val_accuracy: 0.9993
Epoch 8/10
5697/5697 [==============================] - 25s 4ms/step - loss: 0.0052 - accuracy: 0.9994 - val_loss: 0.0055 - val_accuracy: 0.9994
Epoch 9/10
5697/5697 [==============================] - 25s 4ms/step - loss: 0.0060 - accuracy: 0.9994 - val_loss: 0.0063 - val_accuracy: 0.9993
Epoch 10/10
5697/5697 [==============================] - 25s 4ms/step - loss: 0.0054 - accuracy: 0.9994 - val_loss: 0.0055 - val_accuracy: 0.9993


## https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/structured_data/feature_columns.ipynb#scrollTo=_YJPPb3xTPeZ